# drawio file updater

The [drawio](https://github.com/jgraph/drawio) distribution, submoduled into `./drawio`, is very large. We need to distribute parts of it, so that they are available on end user's computers.

## Finding the files

In [1]:
from pathlib import Path
import re, json, IPython

In [2]:
ROOT = Path.cwd().parent
SRC =  ROOT / "src"
DIO = ROOT / "drawio"
DIO_APP = DIO /  "src" / "main" / "webapp"
ALL_FILES = sorted(DIO_APP.rglob("*.*"))
WANT_FILES = set()

def show_files():
    IPython.display.display(IPython.display.Markdown(f"> ### Packaging {len(WANT_FILES)} of {len(ALL_FILES)} files"))
show_files()

> ### Packaging 0 of 3453 files

## The manifest
`drawio` can be shipped as a service worker. 

In [3]:
CACHE_MANIFEST = DIO_APP / "service-worker.js"
manifest = CACHE_MANIFEST.read_text()
WANT_FILES |= {
    DIO_APP / p["url"] for p in 
    json.loads(manifest.split('workbox.precaching.precacheAndRoute(')[1].split(']')[0] + ']')
}
show_files()

> ### Packaging 116 of 3453 files

## Sweep Up
This results in a working app, but emits a few files `404`s... let's manually add some files.

In [4]:
WANT_FILES |= {
    DIO_APP / "export3.html",
    DIO_APP / "js/PreConfig.js",
    DIO_APP / "js/PostConfig.js",
    *(DIO_APP / "math").rglob("*.*"),
    *(DIO_APP / "mxgraph/images").rglob("*.png"),
    *(DIO_APP / "mxgraph/images").rglob("*.gif"),
    *(DIO_APP / "images").rglob("*.svg"),
    *(DIO_APP / "img").rglob("*.svg"),
    *(DIO_APP / "img").rglob("*.png"),
    *(DIO_APP / "images").rglob("*.png"),
    *(DIO_APP / "images").rglob("*.gif"),
    *(DIO_APP / "shapes").rglob("*.js"),
    *(DIO_APP / "stencils").rglob("*.xml"),
    *(DIO_APP / "plugins").rglob("*"),
}
WANT_FILES = [wf for wf in WANT_FILES if not wf.is_dir()]
show_files()

> ### Packaging 2650 of 3453 files

## WTF JSON

Apparently, JSON files are not so good to use with `file-loader`.

In [5]:
WANT_FILES = {wanted for wanted in WANT_FILES if not wanted.name.endswith(".json")}
show_files()

> ### Packaging 2646 of 3453 files

## Get the files into webpack

When launched by JupyterLab, drawio is served in an `iframe`, and all of its assets need to be available locally:

- exactly as they were originally named
- in a predictable location relative to the main `index.html`

This presents some challenges and opportunities:

- Unfortunately, this conflicts with the JupyterLab/WebPack _weltanschauung_ of putting everything into a single file and munging the name with hashes
- Fortunately, webpack's [`file-loader`](https://webpack.js.org/loaders/file-loader/) gives us a way to just copy a file to a predictable location
- Unfortunately, `file-loader` can't handle copying entire folders

So we need to: 
- add the list of files to `package.json`
- tell webpack we actually want the files

In [6]:
PACKAGE_JSON = ROOT / "package.json"

Our `package.json` needs to ship _our_ files, too!

In [7]:
BASE_FILES = [
    "{lib,schema,style}/**/*.{d.ts,eot,gif,html,jpg,js,js.map,json,png,svg,woff2,css}",
    "LICENSE.txt"
]

In [8]:
package_json = json.loads(PACKAGE_JSON.read_text())
package_json["files"] = [*BASE_FILES, *sorted([str(wanted.relative_to(ROOT)) for wanted in WANT_FILES])]
PACKAGE_JSON.write_text(json.dumps(package_json, indent=2, sort_keys=True))

194564

In [9]:
COPY_TMPL = """import '!!file-loader?name=[path][name].[ext]&context=.!../{}';"""

In [10]:
STATIC_TS = SRC / "_static.ts"
STATIC_TS.write_text("""/**
    All files that should be copied to the jupyterlab static folder, available as:
    
    {{:base_url}}static/lab/node_modules/jupyterlab-drawio/src/{{:path}}
    
    This file generated from https://github.com/jgraph/drawio
*/
{}
""".format("\n".join(COPY_TMPL.format(wanted.relative_to(ROOT)) for wanted in sorted(WANT_FILES))))

336339